## Explaination 

I decided to use the XGBoost model with grid search to try and achieve the best possible accuracy as on average XGBoost has yeilded more accurate results than CLF or KNN in my experience. Due to the nature of the dataset I omitted one-hot encoding the features as it would be redundant. 


I opted to utalize the parameters max_depth: [2,4,8], n_estimators: [100, 200],
 colsample_bytree: [0.5, 1.0], because they were parameters that I had used before and decided to try my luck with them in this case. I'm sure that there is room for improvement in my choices however. I would have utalized a more varied set of hyper parameters, however when I attempted to do that was running for well over an hour and I didn't want the runtime to close due to inactivity.

The greatest challange that I had was with manipulating the labels. I had to shift them to a numeric form in range 0-8 to be used in the model. Doing this then presented an issue when printing to the .csv due to incorrect formatting. My method to rectify this was to save the predictions to a dataframe, reverse the numeric shift that I did initally, and then one-hot encode the resulting dataframe. This allowed me to format the data as per the submission guidelines.  

The score that I recieved from kaggle was 5.998 for the public score and 5.949 for the private score. I am unsure as to how these scores are determined however a trend that I noticed was the lower the better, and my score was similar to those in the high 2000-3000 placement range. 


# XGBoost with Grid Search

In [ ]:
import numpy as np 
import pandas as pd
import os
import sklearn

### Parsing and Preprocessing

In [ ]:
train_data = pd.read_csv('/kaggle/input/crate-and-barrel/train.csv')
test_data = pd.read_csv('/kaggle/input/crate-and-barrel/test.csv')

train_data = train_data.set_index('id')
test_data_col = test_data
test_data = test_data.set_index('id')
train_labels = train_data['target'] 
train_features = train_data.drop('target', axis =1)

train_features.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
id,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,6,1,5,0,0,1,...,22,0,1,2,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [ ]:
print(train_labels.unique())

train_labels = train_labels.replace(to_replace="Class_1", value=0)
train_labels = train_labels.replace(to_replace="Class_2", value=1)
train_labels = train_labels.replace(to_replace="Class_3", value=2)
train_labels = train_labels.replace(to_replace="Class_4", value=3)
train_labels = train_labels.replace(to_replace="Class_5", value=4)
train_labels = train_labels.replace(to_replace="Class_6", value=5)
train_labels = train_labels.replace(to_replace="Class_7", value=6)
train_labels = train_labels.replace(to_replace="Class_8", value=7)
train_labels = train_labels.replace(to_replace="Class_9", value=8)


print(train_labels.unique())

['Class_1' 'Class_2' 'Class_3' 'Class_4' 'Class_5' 'Class_6' 'Class_7'
 'Class_8' 'Class_9']
[0 1 2 3 4 5 6 7 8]


### Declare, Fit and Test Model

In [ ]:
from xgboost import XGBClassifier
model1 = XGBClassifier(tree_method = 'gpu_hist', predictor = 'gpu_predictor')

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'max_depth': [2,4,8],
     'n_estimators': [100, 200],
      'colsample_bytree': [0.5, 1.0]}
]

In [ ]:
grid_search= GridSearchCV(model1, param_grid, verbose=4, n_jobs=-1, cv=5)


In [ ]:
result = grid_search.fit(train_features, train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 2/5] END colsample_bytree=0.5, max_depth=2, n_estimators=100;, score=0.774 total time=   5.1s
[CV 4/5] END colsample_bytree=0.5, max_depth=2, n_estimators=100;, score=0.782 total time=   2.9s
[CV 1/5] END colsample_bytree=0.5, max_depth=2, n_estimators=200;, score=0.784 total time=   4.3s
[CV 3/5] END colsample_bytree=0.5, max_depth=2, n_estimators=200;, score=0.791 total time=   4.0s
[CV 5/5] END colsample_bytree=0.5, max_depth=2, n_estimators=200;, score=0.788 total time=   4.2s
[CV 2/5] END colsample_bytree=0.5, max_depth=4, n_estimators=100;, score=0.798 total time=   4.0s
[CV 4/5] END colsample_bytree=0.5, max_depth=4, n_estimators=100;, score=0.806 total time=   4.3s
[CV 1/5] END colsample_bytree=0.5, max_depth=4, n_estimators=200;, score=0.805 total time=   8.4s
[CV 3/5] END colsample_bytree=0.5, max_depth=4, n_estimators=200;, score=0.807 total time=   8.5s
[CV 5/5] END colsample_bytree=0.5, max_depth=4, n_estimat

In [ ]:
grid_search.best_params_

{'colsample_bytree': 0.5, 'max_depth': 8, 'n_estimators': 200}

In [ ]:
pred = grid_search.best_estimator_.predict(test_data)

### Postprocessing of Prediction Data

In [ ]:
#Formatting data for .csv
submission = pd.DataFrame({'id':test_data_col['id'],'pred':pred})

#reverse numeric encoding
submission['pred'] = submission['pred'].replace(to_replace=0, value="Class_1")
submission['pred'] = submission['pred'].replace(to_replace=1, value="Class_2")
submission['pred'] = submission['pred'].replace(to_replace=2, value="Class_3")
submission['pred'] = submission['pred'].replace(to_replace=3, value="Class_4")
submission['pred'] = submission['pred'].replace(to_replace=4, value="Class_5")
submission['pred'] = submission['pred'].replace(to_replace=5, value="Class_6")
submission['pred'] = submission['pred'].replace(to_replace=6, value="Class_7")
submission['pred'] = submission['pred'].replace(to_replace=7, value="Class_8")
submission['pred'] = submission['pred'].replace(to_replace=8, value="Class_9")


#create columns for each classification 

one_hot = pd.get_dummies(submission['pred'])
submission = submission.drop('pred', axis=1)
submission = submission.join(one_hot)


submission

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,1,0,0,0,1,0,0,0,0,0
1,2,0,0,0,0,0,0,0,1,0
2,3,0,0,0,0,0,1,0,0,0
3,4,0,1,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
144363,144364,1,0,0,0,0,0,0,0,0
144364,144365,0,0,1,0,0,0,0,0,0
144365,144366,0,1,0,0,0,0,0,0,0
144366,144367,0,0,0,1,0,0,0,0,0


In [ ]:
submission.to_csv('XGB_Grid_submission.csv', index=False)